In [1]:
from src.utils import is_notebook

%matplotlib widget

ModuleNotFoundError: No module named 'src'

# Module Tests

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from src.plotting import plot_boundaries, plot_particles

## System creator

In [ ]:
# system
from src.system_creator import SystemCreator

# Tube creation :
tube_segments = 0.001*np.array([[0,0,10,0], [0,0,0,1], [10,0,10,1], [0,1,10,1]])
tube = SystemCreator(tube_segments)

offsets = tube.get_offsets()
system_shape = tube.system_shape()
a = tube.get_dir_vects()

print(f'Tube : \noffsets : {offsets}')
print(f'System size : {system_shape} \n')

# Cylinder
circle = [[5+np.cos(k*np.pi/8), 5+np.sin(k*np.pi/8), 5+np.cos((k+1)*np.pi/8), 5+np.sin((k+1)*np.pi/8)] for k in range(16)]
cylinder_segments = 0.001*np.array([[0,0,10,0], [0,0,0,10], [10,0,10,10], [0,10,10,10]]+circle)
cylinder = SystemCreator(cylinder_segments)

print(f'Cylinder : \noffsets : {cylinder.get_offsets()}')
print(f'System size : {cylinder.system_shape()}')

In [ ]:
fig, ax = plt.subplots();
plot_boundaries(ax, tube_segments);

In [ ]:
fig, ax = plt.subplots()
plot_boundaries(ax, cylinder_segments)
plt.show()

## Particle

In [ ]:
from src.utils import Particle, get_mass_part

# Iodine
container = Particle('I', 0, get_mass_part(53, 53, 74), radius = 2e-10, size_array = 10000)
N = 500
arr =  np.random.random((N,5))
print(f'mass, charge, radius, cross-section = {container.get_params()}'); # mass, charge, radius, cross-section

In [ ]:
container.add_multiple(arr)
print(container.get_current())
print(container.get_particles().shape)
print(container.arr.shape);

In [ ]:
print(f'Number of particles : {container.get_current()}')
for k in range(3):
    container.add_multiple(np.random.random((1000,5)))
    print(f'Number of particles : {container.get_current()}')
    container.delete_multiple(np.random.choice(a = container.get_current(), size = 500, replace = False))
    print(f'Number of particles : {container.get_current()}')


In [ ]:
# TODO : make 'dynamic' arrays (when increasing the size).
container.add_multiple(np.random.random((3000,5)))
print(f'Number of particles : {container.get_current()}')

**Conclusion**  : make 'dynamic' array (can increase time, but not decrease it for example).

## Grid
For now : 2D-ndarray with value of type *ndarray* which are 2D.
```python
# grid[x_int, y_int] is the container for a cell
# grid[x_int, y_int][idx] contains the particle indexes
grid[x_int, y_int][idx] = [idx_container, idx_particle_in_container]
```

In [ ]:
from src.utils import Grid, pos_in_grid
resolutions = np.array([4,4])
max_number_per_cell = 10 # to initialize the array which will contain the particles indexes

In [ ]:
grid = Grid(resolutions, max_number_per_cell)

In [ ]:
# x, y, vx, vy, vz
arr = np.array([[-1.5,1.2,1,0,0], [-1.3,-0.8,0,-1,0], [1.3,-0.5,-1,0,0], [1,1,-1,-1,-1], [-1,1,1,0,0]])
print(arr);

In [ ]:
# signature : pos_in_grid(pos, grid_res, offsets, system_shape)
pos = pos_in_grid(arr[:,:2], resolutions, offsets = np.array([-2, -2]), system_shape = np.array([4,4])) # not inplace
print(pos);

In [ ]:
# [pos_x, pos_y, idx_container, idx_particle_in_container]
idx_container = np.array([0,0,0,0,0])
idx_particle_in_container = np.array([0,1,2,3,4])
idxes = np.stack((idx_container, idx_particle_in_container), axis = 1)
new_arr = np.concatenate((pos, idxes), axis = 1)
print(new_arr);

In [ ]:
grid.reset()
grid.add_multiple(new_arr)
print(grid.current);
print(grid.get([0,1]));

In [ ]:
# TODO : delete multiple
grid.delete([0,1], 0) # can still take lots of time
print(grid.current);
print(grid.get([0,1]));

**Conclusion** :
- Add a 'delete_multiple' built on *NumPy* - **DONE**
- Maybe change the grid to a 4D-array :
    - Pro : faster everything.
    - Cons : waste of memory for system with density gradients, increasing size of the array costs much more.

## Injector 

In [ ]:
 # notebook
from src.plotting import plot_particles, plot_boundaries
import src.plotting.analysis as analysis
from src.utils import inject 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Signature : inject(in_wall, in_vect, debit, vel_std, radius, dt)
in_wall =  np.array([0,0,0,1]) # np.array([0,0,0,1]) # np.array([0,0,1,0]) # np.array([0,0,0,1]) # np.array([0,0,1,1])
a = np.array([0,1,1])
in_vect = np.array([1,0]) # (1/np.sqrt(2))*np.array([-1,1]) # -np.array([1,0]) #  -np.array([0,1]) # np.array([1,0])# (1/np.sqrt(2))*np.array([1,1])
debit = 100000000 # particles / s
dt = 0.001
vel_std = 200. # m/s
radius = 0.01

In [ ]:
arr, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt)

In [ ]:
fig, ax = plt.subplots()
plot_boundaries(ax, np.expand_dims(in_wall, axis = 0))
plot_particles(ax, arr, r = 10*radius, arrows = False)
plt.axis('equal');

In [ ]:
df = pd.DataFrame(arr, columns = ['x', 'y', 'vx', 'vy', 'vz'])
df

In [ ]:
analysis.velocity_distribution(df, bins = 50);

## Advection (and schemes)

In [ ]:
from src.utils import advect
from src.utils import euler_explicit, leap_frog
from src.plotting import plot_particles, plot_boundaries

# notebook
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# signature : advect(arr, f, dt, args, scheme)
arr = np.array([[1,1,1,1,0], [2,1,-1,-1,0]], dtype = float)
#arr = np.random.random((10,5))
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))

In [ ]:
arr_ = np.copy(arr)
advect(arr_, f, dt = 0.1, args = [], scheme = euler_explicit) # seems ok

In [ ]:
arr__ =  np.copy(arr)
advect(arr__, f, dt = 0.1, args = [], scheme = leap_frog) # seems ok

In [ ]:
fig, ax = plt.subplots()
plot_particles(ax, arr, r = 10) 
plot_particles(ax, arr_, r = 10)

## Collision with walls

In [ ]:
from src.plotting import plot_boundaries, plot_particles
import matplotlib.pyplot as plt


In [ ]:
from src.utils import handler_wall_collision, make_collisions, make_collisions_vectorized, make_collisions_out_walls
import numpy as np
N = 10
walls = 1.5*np.array([[0,0,1,0], [0,0,0,1], [1,0,1,1], [0,1,1,1]]) # bottom, left, right, top
a = np.array([[1,0, 1.5],[0,1, 1.5],[0,1, 1.5],[1,0, 1.5]])
# arr = np.array([[2,0.5,1,0,0], [0.5,2,0,1,0], [2,2,1,1,0], [0.5,0.5,1,0,0], [-0.5,0.5,-1,0,0]])  # np.random.random((N,5)) # 
arr = np.array([[1.5,2,1,1,0]])
arr[:,2:] = 5*arr[:,2:]
radius = 0.1
idx_out_walls = [2]

In [ ]:
fig, ax = plt.subplots()
plot_boundaries(ax, walls)
plot_particles(ax, arr, r = 8, arrows = True)
plt.axis('equal');

In [ ]:
ct, cp = handler_wall_collision(arr, walls, a, radius)

In [ ]:
new_arr_1 = np.copy(arr)
new_arr_2 = np.copy(arr)
new_arr_3 = np.copy(arr)
make_collisions(new_arr_1, a, ct, cp)
make_collisions_vectorized(new_arr_2, a, ct, cp)
indexes = make_collisions_out_walls(new_arr_3, a, ct, cp, idx_out_walls)
print(indexes)

In [ ]:
fig, ax = plt.subplots(2,2)

plot_boundaries(ax[0,0], walls)
plot_particles(ax[0,0], arr, r = 8, arrows = True)

plot_boundaries(ax[0,1], walls)
plot_particles(ax[0,1], new_arr_1, r = 8, arrows = True)

plot_boundaries(ax[1,0], walls)
plot_particles(ax[1,0], new_arr_2, r = 8, arrows = True)

plot_boundaries(ax[1,1], walls)
plot_particles(ax[1,1], new_arr_3, r = 8, arrows = True)

ax[0,0].axis('equal')
ax[0,1].axis('equal')
ax[1,0].axis('equal')
ax[1,1].axis('equal')
fig.tight_layout();

#### Collision with walls - and outwalls

Problem : sometimes a particle can collide and be reflected but remain outside the system. In such a case, we have to make sure the particle is reflected again, and as many time as necessary. In addition, particles that went out (by the out walls) should be taken into account and :
 - not reflected back into the system but instead removed
 - a particle can, after its second reflection in a row, finds itself going trough the 'out wall'. In such a case it should be added to the list of particle to delete.

In [ ]:
from src.utils import handler_wall_collision, make_collisions, make_collisions_vectorized, make_collisions_out_walls
import numpy as np
from src.plotting import plot_boundaries, plot_particles
import matplotlib.pyplot as plt

N = 10
segments = 1.5*np.array([[0,0,1,0], [0,0,0,1], [1,0,1,1], [0,1,1,1]]) # bottom, left, right, top
a = np.array([[1,0, 1.5],[0,1, 1.5],[0,1, 1.5],[1,0, 1.5]])
arr = np.array([[-4,5.5,-1,1,0], [1.6,2,1,1,0],[2,0.5,1,0,0], [2,2,1,1,0], [0.5,0.5,1,0,0], [-0.5,0.5,-1,0,0]])  # np.random.random((N,5)) # 
radius = 0.01
idx_out_walls = [2] # 2 : Right
fig, ax = plt.subplots()
plot_boundaries(ax, segments)
plot_particles(ax, arr, r = 8, arrows = True)
plt.axis('equal');

In [ ]:
count = np.full(shape = (arr.shape[0]), fill_value = True)
idxes_out = []
c = 0
while(np.sum(count, where = count == True) > 0):
        print(c)
        c+=1
        ct, cp = handler_wall_collision(arr[count], segments, a, radius)
        print(cp)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
        fig, ax = plt.subplots()
        plot_boundaries(ax, segments)
        plot_particles(ax, arr, r = 8, arrows = True)
        plt.axis('equal');
        
        #print(count)
        if(c>20):
            break
print(c)
idxes_out = np.concatenate(idxes_out)

arr[idxes_out.shape[0]:,:] = np.delete(arr, idxes_out, axis = 0) # operation is not inplace
c-=idxes_out.shape[0]

### Collision with walls - cylinder - why does it not work 

In [ ]:
from src.utils import handler_wall_collision, make_collisions, make_collisions_vectorized, make_collisions_out_walls, deal_with_corner
import numpy as np
from src.plotting import plot_boundaries, plot_particles
import matplotlib.pyplot as plt
from src.system_creator import SystemCreator

In [ ]:
res = 4
circle = [[1.5+0.5*np.cos(k*np.pi/res), 1+0.5*np.sin(k*np.pi/res), 1.5+0.5*np.cos((k+1)*np.pi/res), 1+0.5*np.sin((k+1)*np.pi/res)] for k in range(2*res)]
segments = 0.001*np.array([[0,0,3,0], [0,0,0,2], [3,0,3,2], [0,2,3,2]]+circle)
system = SystemCreator(segments)

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

fig, ax = plt.subplots()
plot_boundaries(ax, segments, color = 'k')

# liste_vectors_segment = np.concatenate((segments[:,:2], segments[:,:2]+0.001*a[:,:2]), axis = 1)
# plot_boundaries(ax, liste_vectors_segment, color = 'r')

In [ ]:
# defining particles
# arr = 0.001*np.array([[-4,5.5,-1,1,0], [1.6,2,1,1,0],[2,0.5,1,0,0], [2,2,1,1,0], [0.5,0.5,1,0,0], [-0.5,0.5,-1,0,0]])  # np.random.random((N,5)) # 
arr = np.array([[0.0015, 0.001, 100, 0,0]])
radius = 1e-6

In [ ]:
count = np.full(shape = (arr.shape[0]), fill_value = True)
idxes_out = []
c = 0
idx_out_walls = []
while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision(arr[count], segments, a, radius)
        print(ct)
        deal_with_corner(ct)
        print(ct)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)
        fig, ax = plt.subplots()
        plot_boundaries(ax, segments)
        plot_particles(ax, arr, r = 1, arrows = False)
        plt.axis('equal');

        if(c>20):
            break

np.concatenate(idxes_out)

# arr[idxes.shape[0]:,:] = np.delete(arr, idxes, axis = 0) # operation is not inplace
# current-=idxes.shape[0]

## Particles collisions

Signatures of the functions :
* candidates(currents, dt, average, pmax, volume_cell, mr, remains)
* index_choosen_couples(current, candidates)
* probability(vr_norm, pmax, cross_sections)
* is_colliding(proba)
* reflect(arr, vr_norm)


In [ ]:
from src.utils import candidates, index_choosen_couples, probability, is_colliding, reflect
import numpy as np
from src.plotting import plot_particles, plot_grid
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
currents = 10*np.array([
    [16,22,14],
    [9,13,11],
    [4,9,6]
])
dt = 1e-7
averages = 10*np.array([
    [15,20,15],
    [10,15,10],
    [5,10,5]
])
radius = 2e-10
cross_section = 4 * np.pi * radius**2
pmax = cross_section * 600
pmax_vect = cross_section * 600 * np.ones(averages.shape)
print(pmax_vect.shape)
volume_cell = (0.001)**3
mr = 1e15
remains = 0

In [ ]:
remains, cands = candidates(currents, dt, averages, pmax, volume_cell, mr, remains)


In [ ]:
remains, cands = candidates(currents, dt, averages, pmax_vect, volume_cell, mr, remains)


In [ ]:
arrays = [np.random.uniform(low = -1, high = 1, size = (current, 5)) for current in currents.flatten()]
arr = np.concatenate(arrays, axis = 0)
arr_save = np.copy(arr)
print(arr.shape);

In [ ]:

grid_shape = currents.shape
system_shape = (2.,2.)
offsets = np.array([-1, -1])
fig, ax = plt.subplots()
plot_grid(ax, grid_shape, system_shape, offsets)
plot_particles(ax, arr, r = 0.1)
plt.axis('equal')
plt.show()

In [ ]:
for count, (i, j) in enumerate(np.ndindex(currents.shape)):
    choice = index_choosen_couples(currents[i,j], int(cands[i,j]))
    vr_norm = np.linalg.norm((arrays[count][choice][:,1,2:]-arrays[count][choice][:,0,2:]), axis = 1)
    proba = probability(vr_norm = vr_norm, pmax = pmax, cross_sections = cross_section)
    collidings_couples = is_colliding(proba)
    print(np.sum(collidings_couples))
    # if(not all(~collidings_couples)):
    #    ic(all(~collidings_couples))
    #    ic(arr)
    # can not be inplace
    arrays[count][choice[collidings_couples]] = reflect(arrays[count][choice[collidings_couples]], vr_norm[collidings_couples])


In [ ]:
import pandas as pd

In [ ]:
arr = np.concatenate(arrays, axis = 0)
df1 = pd.DataFrame(arr_save, columns = ['x','y','vx','vy','vz'])
df2 = pd.DataFrame(arr, columns = ['x','y','vx','vy','vz'])

In [ ]:
fig, ax = plt.subplots(2,2)
df1['vx'].plot.hist(bins=50, ax = ax[0,0])
df1['vy'].plot.hist(bins=50, ax = ax[0,1])
df2['vx'].plot.hist(bins=50, ax = ax[1,0])
df2['vy'].plot.hist(bins=50, ax = ax[1,1])

In [ ]:
import seaborn as sns
sns.set_theme(style="white")

#fig, ax = plt.subplots()
g = sns.JointGrid(data=df1, x="vx", y="vy", space=0)
g.plot_joint(sns.kdeplot,
             fill=True,
             thresh=0, levels=100, cmap="rocket")
g.plot_marginals(sns.histplot, color="#03051A", alpha=1, bins=25)

In [ ]:
import seaborn as sns
sns.set_theme(style="white")

#fig, ax = plt.subplots()
g = sns.JointGrid(data=df2, x="vx", y="vy", space=0)
g.plot_joint(sns.kdeplot,
             fill=True,
             thresh=0, levels=100, cmap="rocket")
g.plot_marginals(sns.histplot, color="#03051A", alpha=1, bins=25)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

x = df1['x']
y = df2['y']

# Calculate the point density
xy = np.vstack([x,y])
z = gaussian_kde(xy)(xy)

fig, ax = plt.subplots()
ax.scatter(x,y, c=z, s=100)
plt.show()

### From grid, get particles



In [ ]:
import numpy as np
from src.utils import candidates, index_choosen_couples, probability, is_colliding, reflect

currents = 10*np.array([
    [24,26],
    [25,25]
], dtype = int)

arr = [np.random.uniform(low = -1.0, high = 1.0, size = (1000,5))]
arr_copy = np.copy(arr[0])
# zer = np.zeros(250)
t = np.arange(1000)
np.random.shuffle(t)
arr1, arr2, arr3, arr4 = np.split(t, indices_or_sections = [240,500,750], axis = 0)
p1, p2 = np.stack((np.zeros(240), arr1), axis = 1), np.stack((np.zeros(260), arr2), axis = 1)
p3, p4 = np.stack((np.zeros(250), arr3), axis = 1), np.stack((np.zeros(250), arr4), axis = 1)

grid = np.array([
    [p1, p2],
    [p3, p4]],
    dtype = np.ndarray)

print(f'Currents : \n {currents}');

In [ ]:
# Now we have to get all that is in arr from grid...
# and do this in place ... which will be VERY fucking hard
# VERY SLOW
cands = np.array([[120,123],[124,124]])
cross_section = 1
pmax = 2
for k, (i, j) in enumerate(np.ndindex(currents.shape)):
    choice = index_choosen_couples(currents[i,j], int(cands[i,j])) # choice contains the possible couples
    # now, we have to get their pos and speed
    g = grid[i,j]
    parts = np.array([[g[c[0]], g[c[1]]] for c in choice], dtype = int)
    array = np.array([[ arr[c[0,0]][c[0,1]] , arr[c[1,0]][c[1,1]] ] for c in parts])
    # blablabla [...] -> DSMC etc.
    # TODO: make those stuff and see if it still works

    vr_norm = np.linalg.norm((array[:,1,2:]-array[:,0,2:]), axis = 1)
    proba = probability(vr_norm = vr_norm, pmax = pmax, cross_sections = cross_section)

    # TODO : should update pmax here (or return something)...
    collidings_couples = is_colliding(proba)
    array[collidings_couples] = reflect(array[collidings_couples], vr_norm[collidings_couples])
    
    for k in range(len(array)):
        c1, c2 = array[k,0], array[k,1]
        c = parts[k]
        arr[c[0,0]][c[0,1]][:] = c1 # copy
        arr[c[1,0]][c[1,1]][:] = c2
        

In [ ]:
np.array_equal(arr_copy, arr[0])

### Reflection study 

In [ ]:
from src.utils import reflect
import numpy as np


arr = np.array([[[1,1,1,1,1], [1,1,-1,-1,1]]])
vr_norm = np.linalg.norm((arr[:,1,2:]-arr[:,0,2:]), axis = 1)

In [ ]:
arr_ = reflect(np.copy(arr),vr_norm)

In [ ]:
print(arr);
print(arr_);

## Update position in grids - tests

##### Idea : 
1. Create particles and initialize them in the grids
2. Do something 
3. Update position in the grid

For now, we are simply resetting all of them as it does not cost much to do so. (structured grid)

In [ ]:
from src.utils import Grid, pos_in_grid
import numpy as np
import matplotlib.pyplot as plt

N = 100
arr = np.random.uniform(low = -1, high = 1, size = (N,5))
grid = Grid(np.array([3,7]), 10)

from src.plotting import plot_particles, plot_grid
resolutions = (3,7)
system_shape = (2.,2.)
offsets = np.array([-1, -1])
fig, ax = plt.subplots()
plot_grid(ax, resolutions, system_shape, offsets)
plot_particles(ax, arr, r = 0.1)
plt.axis('equal')
plt.show()

In [ ]:
def convert_to_grid_datatype(positions, new, old = 0):
    index_container = np.zeros((new-old))
    index_in_container = np.arange(old, new)
    indexes = np.stack((index_container, index_in_container), axis = 1)
    return np.concatenate((positions, indexes), axis = 1).astype(int)

In [ ]:
positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
particles = convert_to_grid_datatype(positions, new = positions.shape[0])
print(particles[:10])
grid.add_multiple(particles)
print(grid.current);

In [ ]:
grid.reset() # that's how we are going to do...
print(grid.current);
grid.add_multiple(particles)
print(grid.current);

# Integration test

Signatures :
 - SystemCreator(segments)
 - inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
 - advect(arr, f, dt, args, scheme)
 - handler_wall_collision(arr, walls, a, radius)
 - make_collisions_vectorized(arr, a, ct, cp)
 - Particle(part_type, charge, mass, radius, size_array)
 - Grid(resolutions, max_number_per_cell)
 - collider(arr, grid, currents, dt, average, pmax, cross_section, volume_cell, mr, remains)

In [ ]:
%matplotlib widget

# system
from src.system_creator import SystemCreator

# Grid
from src.utils import Grid, pos_in_grid, convert_to_grid_datatype

# Particles
from src.utils import Particle

# injection 
from src.utils import inject

# advection
from src.utils import advect
from src.utils import euler_explicit, leap_frog

# collisions
from src.utils import handler_wall_collision, handler_wall_collision_point, make_collisions_vectorized, make_collisions_out_walls, deal_with_corner

# utils 
from src.utils import gaussian, maxwellian_flux, maxwellian_mean_speed, get_mass_part, mean_free_path, mean_free_time

# systems
from src.utils import systems

# plotting 
from src.plotting import plot_boundaries, plot_particles, plot_grid, plot_system
from src.plotting import analysis

# collisions between particles
from src.utils import handler_particles_collisions, candidates # candidates, index_choosen_couples, probability, is_colliding, reflect, 

# saving 
from src.data import Saver

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import os
np.random.seed(1111)

In [ ]:
# System :
dz = 0.001

    # tube, square etc.
system, idx_out_walls, idx_in_wall = systems.system_rectangle(lx = 0.01, ly = 0.001)

    # thruster 
# dp = 0.001
# isystem, idx_out_walls, idx_in_wall = systems.thruster_system(w_in = 5*dp, l_in = 3*dp, w1 = 3*dp, l1 = dp, l_int = dp, w2 = dp, l2 = 5*dp, w_out = 5*dp, l_out = dp, offsets = np.array([0,0]))

    # cylinder
# system, idx_out_walls, idx_in_wall = systems.cylinder_system(res = 4, lx = 0.003, ly = 0.001, cx = 0.0015 , cy = 0.0005, r = 0.0001)

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

# grid :
mean_number_per_cell = 1000
max_number_per_cell = 10*mean_number_per_cell
resolutions = np.array((10,1), dtype = int) # tube
# resolutions = np.array((11,5), dtype = int) # thruster
# resolutions = np.array((9,9), dtype = int) # cylinder

grid = Grid(resolutions, max_number_per_cell)
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]

# Particles - 1 type 
density = 3.2e19 # m-3
n_simu = mean_number_per_cell*np.prod(resolutions) # number of particles in the simulated system
n_real = volume_cell * density * np.prod(resolutions) # number of particles in the real system
mr = n_real/n_simu # macro particules ratio = number of particles in the real system / number of macro part in the simulated system
density_dsmc = density/mr
temperature = 300 # K

part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10
size_array = 2*mean_number_per_cell*np.prod(resolutions)
v_mean = maxwellian_mean_speed(temperature, mass)
container = Particle(part_type, charge, mass, radius, size_array)
cross_section = container.get_params()[3]

# mean free path and time
mfp = mean_free_path(cross_section, density)
typical_lenght = 0.001
mft = mean_free_time(typical_lenght, v_mean = v_mean)

    # Injection params
in_wall = segments[idx_in_wall]
# you may have to change in_vect (adding a '-' should be enough) depending on the direction of injection of the particles
in_vect = np.array([a[idx_in_wall,1], -a[idx_in_wall,0]]) # np.array([in_wall[3]-in_wall[1],in_wall[0]-in_wall[2]]) # a[idx_in_wall]

debit = maxwellian_flux(density_dsmc, v_mean)*np.linalg.norm(in_wall[:2]-in_wall[2:])*dz
vel_std = gaussian(temperature, mass)

In [ ]:
# Simulation params
iterations = 1000
dt = 1e-6 # in sec, should be a fraction of the mean free time

# saving params
saving_period = 10
adding_period = 1

# advection
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))

args = []
scheme = euler_explicit

In [ ]:
print(v_mean);
print(mfp);
print(mft);
print(debit*dt);
print("{:e}".format(n_real))
print("{:e}".format(mr));
print(vel_std)
print(v_mean)
new, remains = inject(in_wall, in_vect, debit*100, vel_std, radius, dt, 0)
print(np.mean(np.linalg.norm(new[:,2:], axis = 1)))

In [ ]:
plot_system(None, segments, radius, resolutions, system_shape, offsets);

In [ ]:
# NAME tests
from pathlib import Path

dir_path = Path('results/')
name = 'test_tube.h5'

saver = Saver(dir_path, name)

In [ ]:
df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # bucket for the particles - index of particles is the iteration number

# defining useful arrays and ints 
remains = 0 # fractionnal part of the number of particles to inject (it is then passed to the following time step)
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell) # average number of particles per cell
pmax = 2*v_mean*cross_section*np.ones(averages.shape) # max proba per cell in the simu
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float) # remains per cell for the particles collisions step

# SIMULATING
print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(iterations): # tqdm
    n1 = container.get_current()
                   
    # injecting particles
    new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    if(it%adding_period == 0):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    collisions_with_walls = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        # tracking then number of wall collisions
        if(c == 1):
            collisions_with_walls = np.sum(count, where = count == True)
        # the first value of np.sum(count, where = count == True) is then number of colliding particles !
        idxes_out.append(idxes_out_)
    
    idxes_out = np.concatenate(idxes_out)
    
    # TODO : make delete multiple better - currently the function creates a new array where as we can do it inplace.
    container.delete_multiple(idxes_out)
    
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
        # TODO : change the way it's done
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)
        
        # DSMC
        # TODO: make parallel
    currents = grid.get_currents()
    averages = (it*averages+currents)/(it+1) # may be it too violent ? 
    
    remains_per_cell, nb_colls, pmax, monitor = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell, monitoring = True)
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%saving_period==0 or it == iterations-1): # saving if last iterations too
        saver.save(it = it, append = {
                        'df' : df,
                        'collisions_per_cell' : nb_colls, # evolution of the number of collisions per cell - size : grid.shape[0] x grid.shape[1] (2D)
                        'total_distance' : float(monitor[0]), # evolution of the sum of the distance accross all cells 
                        'total_proba' : float(monitor[1]), # evolution of the sum of proba accross all cells
                        'pmax_per_cell' : pmax,  # evolution of the sum of pmax - per cell (2D)
                        'total_deleted' : len(idxes_out), # evolution of the number of deleted particles per cell (int)
                        'averages_per_cell' : averages,
                        'collisions_with_walls' : collisions_with_walls
                  })
        
        # resetting dataframe to not use too much memory
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz'])
        print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(it, n1, n2, idxes_out.shape[0], c))
saver.close()

In [ ]:
%matplotlib widget

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.data import Saver
from src.plotting import analysis
dir_path = Path('results/')
name = 'test_tube.h5'
store = pd.HDFStore(dir_path/name)

In [ ]:
collisions_per_cell = store['collisions_per_cell'] 
df = store['df']
pmax_per_cell = store['pmax_per_cell']
averages_per_cell = store['averages_per_cell']
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']
collisions_with_walls = store['collisions_with_walls']

In [ ]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
frames = unique_index[int(0.8*nb_save):nb_save]

In [ ]:
print(frames)

## Number of particles - evolution

In [ ]:
fig, ax = plt.subplots()
analysis.nb_particles_evolution(ax, store['df'])

## Distribution

In [ ]:
fig, ax = plt.subplots()
analysis.state(ax, df.loc[df.index == 999], c = None)
# analysis.velocity_distribution(df, frames);

In [ ]:
analysis.spatial_hist2d(df, frames, val = 'vx', x_res = 10, y_res = 1, x_step = 0.001, y_step=0.001);

## Number of collisions between particles

In [ ]:
nb_collisions_per_cell = store['collisions_per_cell']
nb_collisions_per_cell = nb_collisions_per_cell.groupby(nb_collisions_per_cell.index).sum()

In [ ]:
fig, ax = plt.subplots()
plt.plot(nb_collisions_per_cell) 

In [ ]:
print('Total number of collision : \n', np.sum(nb_collisions_per_cell));

## Number of collisions with walls 

In [ ]:
fig, ax = plt.subplots()
plt.plot(collisions_with_walls)

## DSMC - monitoring 

- proba (over the system - not by cell)
- distance between collisioning particles (over the system - not by cell)
- average number of particle per cell (by cell)
- pmax (by cell)

In [ ]:
proba = total_proba/nb_collisions_per_cell
dist = total_distance/nb_collisions_per_cell
print('Mean proba : {:e}'.format(np.mean(proba)))
print('Mean distance : {:e} m'.format(np.mean(dist)))

fig, ax = plt.subplots(2)
ax[0].plot(proba.index, proba, label = 'proba')
ax[1].plot(dist.index, dist, label = 'distance');

In [ ]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum()
averages = averages_per_cell.groupby(averages_per_cell.index).sum()
fig, ax = plt.subplots(2)
ax[0].plot(pmax, label = 'proba')
ax[1].plot(averages, label = 'averages')
plt.show()

In [ ]:
df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2)

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
# plt.style.use('seaborn-pastel')

def update_hist(num, df):
    #plt.cla() # to clear the current figure
    dfit = df.loc[df.index == num]
    # since we modifying scat we dont want to use plt.cla
    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    # scat.set_array(df['speed_norm'])
    # plot_grid(ax, resolutions, system_shape)
    ax.set_title('{}/{}'.format(num+1, 1000), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['speed_norm'], cmap='seismic') #  c = df['speed_norm']
plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)
ax.set_title('{}/{}'.format(1, 1000), fontsize=12)

# ax.axis('equal')
# ax.set_xlim(x_min, x_max)
# ax.set_ylim(y_min, y_max)

ax.axis('equal')
# min_x, min_y, max_x, max_y = min(df['x']), min(df['y']), max(df['x']), max(df['y'])
# ax.set(xlim=(-0.001, 0.011), ylim=(-0.0001, 0.0011))
# ax.set(xlim=(0, 0.003), ylim=(0, 0.002))

interval = 40 # 25 images per second

anim = FuncAnimation(fig, update_hist, interval=interval, frames=1000, fargs=(df, ), save_count=1000)
# plt.show()
anim.save('system_evo_thruster_test.mp4', dpi = 300)

## Reactions - collision with walls

In [ ]:
import lppydsmc as ld
import numpy as np

In [ ]:
arr = np.array([[1.2,3.2,1,2,0], [1.2,3.2,9,1,10], [1.2,3.2,4,6,4], [1.2,3.2,1,2,1]])
count = np.array([2, 0, 1, 1])

law = lambda part, c : (0.5)**c # part = [x,y,vx,vy,vz]

In [ ]:
reacts = ld.advection.reactions.basic(arr, count, law)

In [ ]:
reacts

In [ ]:
draw = np.random.random(10)
proba = np.random.random(10)

In [ ]:
np.where(proba>draw)

## Adding reactioons betweeen particles

Following this [article](https://doi.org/10.1080/00268976.2019.1602740). The goal is to make a "background gas" (made of $[I]$), interacting with it (collisions and reactions) and follow two species : the ion $[I^-]$ and the Iodine $[I]$.

To simplify the approach, the background gas will not be updated.

What needs to be implemented is :
1. The background gas
2. The collisions with it
3. The potential reactions resulting from these collisions

In [ ]:
import numpy as np

In [ ]:
# for particle sent ONE by ONE.
# which is why its okay doing it this way.

class BackgroundGas(object):
    # compatible with as many species as we want
    # densities - shape : number of species x number of cells
    # or : number of cells x number of species (this has the advantage that we are iterating over the cells)
    # which would result in better performance most likely.
    # this is easy to revert though
    
    # second question is : how to take into acccount the functional / non functional forms ? (and use the efficiently).
    
    def __init__(self, densities, velocities, species, diameters, frequency_update = 10):
        # we are considering that *number_of_velocities* is the same for every specie, and direction.
        self.densities = densities # shape : number of cells x number of species - ndarrays of floats
        self.dynamics = velocities # shape : number of cells x number of species x 3 - ndarray of object of type DistributionSampler

        self.frequency_update = frequency_update # for now it is not used
        self.species_dict = species # e.g. : {'I': 1, 'I-' : 2}
        self.diameters = diameters # e.g. : {'I':2e-10, } # in meter

    def probability(self, position, species, diameter, vel, dt, approx = None):
        density = self.densities[position, self.species_dict[species]]
        if(diameter == None):
            cross_section = np.pi * self.diameters[self.species_dict[species]]
        else:
            cross_section = 0.5 * np.pi * (self.diameters[self.species_dict[species]]+diameter)
            
        if(approx == None):
            # to do - a case without approximation
            # vel_norm = ...
            vr = np.linalg.norm(vel) # this is wrong
            pass
        elif(approx == 'faster'):
            # vr ~ v_particle
            vr = np.linalg.norm(vel)
            # in this cas, the particle colliding is supposed to be much faster than the backgroung gas
        elif(approx == 'slower'):
            # vr ~ v_background_gas
            vr = self.dynamics[position, self.species_dict[species]].mean_speed()
        elif(approx == 'equal'):
            # vr = vr_background_gas
            vr = self.dynamics[position, self.species_dict[species]].mean_relative_speed()

        return self.proba_collision(density, cross_section, vr, dt)
    
    # probability of collision with each gaz 
    def proba_collision(self, density, cross_section, vr, dt): # the one used in the article by Schullian - it is an approximation though (for small values of V)
        return density * cross_section * vr * dt
    
    def update(self):
        # should update each distribution, and the proba computed with those
        pass
    
class DistributionSampler(object):
    
    def __init__(self, params, functional = True):
        
        self.params = params
        self.functional = functional
        if(functional):
            self.random_variates = self.params
        else:
            self.current = 0
            self.number_of_samples = self.params.shape[0]
            self.random_variates = lambda size : np.choice(self.params, size=size, replace=True, p=None) # self.params should be 1D, if not this function should fails
        
    def draw(self, size = 1):
        return self.random_variates(size)
    
    def update(self, new):
        if(self.functional):
            print("You cannot update a distribution which was given in a functional form.")
        else:
            # replacing the k "older" ones by k more recent ones ; k = new_params.shape[0]
            next_current = self.current + new.shape[0]
            if(next_current > self.number_of_samples):
                self.params[self.current:] = new[:self.number_of_samples-self.current]
                next_current %= self.number_of_samples
                self.params[:next_current] = new[self.number_of_samples-self.current:]
            else:
                self.params[self.current:next_current] = new
                
            self.current = next_current%self.number_of_samples
    
    

In [ ]:
# for a grid of 3 cells
# we will make 4 distributions basically
# considering it the same for vy, vz, for all cells
# and that there is a different drift added in each cell along vx
vx_distrib_form = lambda drift, std : (lambda size: np.random.normal(loc = drift, scale = std, size=size))
vx1_distrib = DistributionSampler(params = vx_distrib_form(100, 200))
vx2_distrib = DistributionSampler(params = vx_distrib_form(120, 200))
vx3_distrib = DistributionSampler(params = vx_distrib_form(110, 200))
vyz_distrib = DistributionSampler(params = vx_distrib_form(0, 200))
densities = np.array([[3.2e19],[3.0e19],[2.5e19]], dtype = float)
velocities = np.array([[vx1_distrib,vyz_distrib,vyz_distrib],[vx2_distrib,vyz_distrib,vyz_distrib],[vx2_distrib,vyz_distrib,vyz_distrib]], dtype = DistributionSampler)
species = ['I']

In [ ]:
background_gas = BackgroundGas(densities = densities, velocities = velocities, species = species) 

In [ ]:
# PROBABILITY OF COLLISION

def proba_approx(density, v1, cross_section, dt): # in our case, we will suppose relative velocity ~ v1 (because the ion is much much faster than the background gas)
    return density*np.linalg.norm(v1)*cross_section*dt # proba that the particle collided with the slow background gas of density *density*.

def proba(v1, v2, cross_section, cell_volume, dt):
    return np.linalg.norm(v2-v1)*cross_section*dt/cell_volume

def proba_gas(density, mean_speed_times_cross_section, dt): # the one used in the article by Schullian
    return density * mean_speed_times_cross_section * dt

# Reactions with walls

In [3]:
import lppydsmc.advection.reactions as reactions
import lppydsmc as ld
path = 'walls_reactions.txt'

In [4]:
print(reactions.parse_file(path))

{'I-': {'#': 1, 'reactants': ['I-'], 1: ['I']}, 'I2': {'#': 3, 'reactants': ['I2'], 1: ['I', 'I'], 2: ['I-', 'I+'], 3: ['I+', 'I']}}


In [5]:
import numpy as np
def p(size):
    proba = np.random.uniform(low=0.0, high=1.0, size = size)
    return (proba*p_+1).astype(int)

In [6]:
p_ = 1
p(10)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## testing reactions functions - with walls to start with

In [14]:
idx_reacting_particles = np.array([[1]], dtype = int)

arr1 = np.array([[0,0,3,4,0],[0,0,0,-4,0,]])
arr2 = np.array([[0,0,2,3,0],[0,0,-1,17,9,]])
arrays = [arr1, arr2]

masses = np.array([1,2], dtype = float)

types_dict = {
    'I' : 0,
    'I-' : 1
}

reactions_list = [
    'I- : I'
]
nb_species = 2
types = ['I', 'I-']
reactions_dict = reactions.parse(reactions_list)

In [15]:
reacting_particles, particles_to_add = ld.advection.reactions.react(idx_reacting_particles, arrays, masses, types_dict, reactions_dict['I-'], p = None)

In [16]:
reactions_dict

{'I-': {'#': 1, 'reactants': ['I-'], 1: ['I']}}

In [17]:
reacting_particles

array([[1]])

In [18]:
particles_to_add

{'I': [array([-2., 34., 18.])]}

In [19]:
list_counts = [np.array([]), reacting_particles]

In [22]:
# print(f'Total collision with walls: {collisions_with_walls}')
print('DELETE')
for k in range(nb_species): # here it's only one particle as it is colliding with the wall
    # thus it is easier to delete
    print('{} - {}'.format(types[k], list_counts[k].shape[0]))
    print(f'Deleting {len(list_counts[k])} particles of type {types[k]}')
    # containers[k].delete_multiple(list_counts[k])
    if(types[k] in particles_to_add):
        print('ADDING - {} - {}'.format(types[k], len(particles_to_add[types[k]])))
        # containers[k].add_multiple(np.array(particles_to_add[types[k]]))

DELETE
I - 0
Deleting 0 particles of type I
ADDING - I - 1
I- - 1
Deleting 1 particles of type I-


### Cfg - testing

In [1]:
import lppydsmc as ld
from pathlib import Path
from pprint import pprint

In [2]:
path_to_cfg = "lppydsmc/config/example.ini"

In [3]:
config = ld.config.cfg_reader.read(path_to_cfg)

In [ ]:
pprint(config.dict())

## Testing cfg run module

In [3]:
import lppydsmc as ld
import pandas as pd

In [ ]:
path_to_cfg = "lppydsmc/config/example.ini"
params = ld.run.run(path_to_cfg, save = True)

In [3]:
for key, val in params['setup']['containers'].items():
    print(val)

Container filled at 5 x 345/18000 - Particle I : m = 2.160e-25 kg - q = 0.000e+00 C, r = 2.000e-10 m, cs = 5.027e-19 m2
Container filled at 5 x 175/18000 - Particle I2 : m = 4.320e-25 kg - q = 0.000e+00 C, r = 2.660e-10 m, cs = 8.891e-19 m2
Container filled at 5 x 3/162 - Particle I- : m = 2.160e-25 kg - q = -1.600e-19 C, r = 2.000e-10 m, cs = 5.027e-19 m2
Container filled at 5 x 4/180 - Particle I+ : m = 2.160e-25 kg - q = 1.600e-19 C, r = 2.000e-10 m, cs = 5.027e-19 m2
Container filled at 5 x 0/18 - Particle e- : m = 9.000e-31 kg - q = -1.600e-19 C, r = 2.800e-15 m, cs = 9.852e-29 m2


In [4]:
# quick analysis - just loading the hdf5
path = params['setup']['path']/'monitoring.h5'
print(path)

/home/calot/Documents/projets/lppydsmc/results/cfg_tests/example/monitoring.h5


In [5]:
store = pd.HDFStore(path)

In [6]:
store.keys()

['/dsmc_collisions',
 '/dsmc_tracking',
 '/fluxes',
 '/out_particles',
 '/particles',
 '/wall_collisions']

In [7]:
store['fluxes'].groupby('species').sum()

,in,out
species,,
0.0,160.0,21.0
1.0,88.0,24.0
2.0,0.0,0.0
3.0,12.0,0.0
4.0,0.0,0.0


In [14]:
store['dsmc_collisions']['quantity'].sum()

90720.0

In [9]:
store['out_particles'].groupby('species').count()

,x,y,vx,vy,vz
species,,,,,
0.0,21,21,21,21,21
1.0,24,24,24,24,24


# Computing an approximation of a given volume

In [11]:
import numpy as np
from time import time
import lppydsmc as ld

In [12]:
points = np.array([(0,0),(0,1),(1,2),(1,0)])
samples = [10,100,1000,int(1e4),int(1e5), int(1e6)]

In [ ]:
for sample in samples :
    t1 = time()
    print(ld.utils.estimation.estimate_surface(sample, points)) 
    # works but it is pretty slow - I should see for reusing my algorithms which are vectorized and kind of optimized already
    print('Time : {} s'.format(time()-t1))

In [ ]:
for key, val in 

# Tests with numpy

In [1]:
import numpy as np

In [2]:
arr = np.zeros((2,3))
inject = np.array([10,23])

In [3]:
arr[:,0] = inject

In [4]:
arr

array([[10.,  0.,  0.],
       [23.,  0.,  0.]])

In [5]:
import lppydsmc as ld

In [9]:
debits = np.array([1.108301380e8, 2.13390831e8, 1.10397103987e9])
remains = np.array([0.9, 0.85, 0.2])
dt = 1e-5

In [10]:
print(ld.injection.get_quantity(debits, remains, dt))

(array([0.20138  , 0.75831  , 0.9103987]), array([ 1109,  2134, 11039]))


In [11]:
debits*dt

array([ 1108.30138  ,  2133.90831  , 11039.7103987])

In [17]:
def inject(remains, debits, dt):
    remains[:], inject_qties = ld.injection.get_quantity(debits, remains, dt)

In [18]:
print(remains)
inject(remains, debits, dt)
print(remains)

[0.20138   0.75831   0.9103987]
[0.50276   0.66662   0.6207974]


In [23]:
ld.collision.collider.index_choosen_couples(1000, 10)

array([[  5, 649],
       [702, 705],
       [889, 136],
       [162, 951],
       [153,  93],
       [568, 631],
       [896, 458],
       [954, 130],
       [444,  91],
       [820, 773]])

In [24]:
def groupby_np(X, groups, axis = 0, uf = np.add, out = None, minlength = 0, identity = None):
    if minlength < groups.max() + 1:
        minlength = groups.max() + 1
    if identity is None:
        identity = uf.identity
    i = list(range(X.ndim))
    del i[axis]
    i = tuple(i)
    n = out is None
    if n:
        if identity is None:  # fallback to loops over 0-index for identity
            assert np.all(np.in1d(np.arange(minlength), groups)), "No valid identity for unassinged groups"
            s = [slice(None)] * X.ndim
            for i_ in i:
                s[i_] = 0
            out = np.array([uf.reduce(X[tuple(s)][groups == i]) for i in range(minlength)])
        else:
            out = np.full((minlength,), identity, dtype = X.dtype)
    uf.at(out, groups, uf.reduce(X, i))
    if n:
        return out

In [35]:
nb_species = 3
arr = np.array([[[0,103],[1,20]],[[0,1],[0,13]],[[1,4411],[2,94]],[[2,4411],[0,94]],[[2,4411],[2,94]]])

In [27]:
arr.shape[0]

5

In [33]:
groupby_np(X = arr, groups = np.array([1,0,4,2,5]), axis = 0, uf = np.multiply, out = None, minlength = 0, identity = None)

array([      0,       0,       0,       1,  829268, 1658536])

In [42]:
groups = (arr[:,0,0]*nb_species+arr[:,1,0]) # you would have to sort first !!



In [43]:
groups

array([1, 0, 5, 6, 8])

In [44]:
from time import time

In [ ]:
# first function
# grouping like we want to 
# not something generic

def groups_1(arr, nb_species): # with the loops
    groups = np.zeros(arr.shape[0])
    for k, a in enumerate(arr):
        c1, c2 = arr[0][0], arr[0][1]
        if(c1>c2):
            groups[k] = c1*

In [47]:
nb_species = 10
groups = np.zeros((nb_species, nb_species))
count = 0
for i in range(nb_species):
    for j in range(i, nb_species):
        groups[i,j] = count
        groups[j,i] = count
        count +=1

In [48]:
groups

array([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
       [ 1., 10., 11., 12., 13., 14., 15., 16., 17., 18.],
       [ 2., 11., 19., 20., 21., 22., 23., 24., 25., 26.],
       [ 3., 12., 20., 27., 28., 29., 30., 31., 32., 33.],
       [ 4., 13., 21., 28., 34., 35., 36., 37., 38., 39.],
       [ 5., 14., 22., 29., 35., 40., 41., 42., 43., 44.],
       [ 6., 15., 23., 30., 36., 41., 45., 46., 47., 48.],
       [ 7., 16., 24., 31., 37., 42., 46., 49., 50., 51.],
       [ 8., 17., 25., 32., 38., 43., 47., 50., 52., 53.],
       [ 9., 18., 26., 33., 39., 44., 48., 51., 53., 54.]])

In [52]:
def set_groups(n):
    groups = np.zeros((n, n))
    count = 0
    for i in range(n):
        for j in range(i, n):
            groups[i,j] = count
            groups[j,i] = count
            count +=1
    return groups

# now we want a function that returns the group from the array
def get_groups(arr, groups):
    return groups[arr[:,0,0],arr[:,1,0]]

In [53]:
n = 3
groups = set_groups(n)

In [55]:
get_groups(arr, groups).astype(int)

array([1, 0, 4, 2, 5])

In [ ]:
unique, counts = numpy.unique(a, return_counts=True)

In [57]:
idx = 2
nb_groups = 10
unique =  np.array([0,2,3,4,6,9])
print(idx*nb_groups+unique)

[20 22 23 24 26 29]


# Testing on pandas datasets

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame(columns = ['1','2','3'])
print(type(df)==pd.DataFrame)
print(df.columns)

True
Index(['1', '2', '3'], dtype='object')


In [4]:
arr = np.array([[1,2,3],[4,5,6]])
iteration = 2930
df = df.append(pd.DataFrame(data = arr, index = [iteration]*arr.shape[0], columns = ['1','2','3']))
print(df)

      1  2  3
2930  1  2  3
2930  4  5  6


In [5]:
type(pd.DataFrame(df))

pandas.core.frame.DataFrame

# Angles issues

In [55]:
import numpy as np

# both arctan and arctan2 returns pi/2 (or -pi/2) for input = np.inf.
# wall
wall = np.array([1,0,1,1]) # vertical wall at x = 1, of lenght 1
directing_vector = np.array([0,1])
cTheta, sTheta = directing_vector[0], directing_vector[1]
with np.errstate(divide='ignore', invalid='ignore'):
    theta = np.arctan(np.divide(directing_vector[1],directing_vector[0])) # does not need arctan2 since we are in the 1st and 4th quadrants

# velocity
v = np.array((1.,1.,-1.), dtype = float)
norm = np.linalg.norm(v)
#with np.errstate(divide='ignore', invalid='ignore'):
a = np.arctan2(v[1], v[0]) # arctan2 will not return an error if dividing by zeros. 
b = np.arctan2(v[2], v[0])
# with atan2, both returned angle are in [-pi, pi]
# in theory we would need only this for one of the two, so maybe we could go from arctan2 -> arctan on b which would be in [-pi/2, pi/2]

In [52]:
new_a = a-theta
new_b = b

In [14]:
import numpy as np
a = np.ones((10,3))
x = np.zeros((10,1))

In [15]:
a*x

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [ ]:
drift_wall = 30 # m/s
def reflect_fn(arr, idxes_walls, a, ct, cp): # _specular
    k1, k2 = 2*a[:,0]**2-1, 2*a[:,0]*a[:, 1]

    # velocity after the collision
    arr[:,2] = arr[:,2]*k1+ arr[:,3]*k2   # i.e. : vx = vx*k1+vy*k2
    arr[:,3] = - arr[:,3]*k1+arr[:,2]*k2  # i.e. : vy = -vy*k1+vx*k2

    # new position (we could add some scattering which we do not do there)
    arr[:,0] = cp[:,0]+ct*arr[:,2] # new x pos 
    arr[:,1] = cp[:,1]+ct*arr[:,3] # new y pos
    
    # then we add a given drift to all particles that collided with walls 1 (since this is the top wall)
    top_wall_idxes = np.where(idxes_walls == 1)[0]
    arr[top_wall_idxes,0] = arr[top_wall_idxes,0] + drift_wall # in place
    return arr

# Trying benchmarks

In [1]:
import matplotlib.pyplot as plt

In [2]:
import lppydsmc as ld
from pprint import pprint
# cfg_path = 'benchmarks/couette.ini'
# cfg_path = 'benchmarks/couette.ini'
cfg_path = 'benchmarks/simple_tube.ini'

In [3]:
#config = ld.config.cfg_reader.read(cfg_path)
params = ld.run(cfg_path, save = True)

|    it    |   INIT   |  INJECT  |   DEL    | C. WALLS | C. PARTS |  REACT   | EXEC TIME (s) |
----------------------------------------------------------------------------------------------
|    0     |    0     |    0     |    0     |    0     |    0     |    NA    |       0       |
|    10    |   276    |    28    |    0     |    19    |    2     |    NA    |   2.935e-02   |
|    20    |   552    |    28    |    0     |    65    |    2     |    NA    |   2.987e-02   |
|    30    |   822    |    27    |    2     |    92    |    2     |    NA    |   2.995e-02   |
|    40    |   1054   |    27    |    9     |   127    |    2     |    NA    |   3.118e-02   |
|    50    |   1253   |    28    |    9     |   133    |    3     |    NA    |   3.782e-02   |
|    60    |   1400   |    28    |    15    |   150    |    2     |    NA    |   3.182e-02   |
|    70    |   1524   |    28    |    11    |   161    |    4     |    NA    |   4.153e-02   |
|    80    |   1615   |    27    |    23    |   17

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7f35222f98c0> (for post_execute):


KeyboardInterrupt: 

### Analysis

In [3]:
def temperature_evolution(data, mass):
    from lppydsmc.utils.physics import BOLTZMAN_CONSTANT
    # speed norm
    data['v2'] = data['vx']*data['vx']+data['vy']*data['vy']+data['vz']*data['vz']
    data['v'] = np.sqrt(data['v2'])

    # drift 
    v_mean = data.groupby(data.index).mean()
    v_mean['drift2'] = v_mean['vx']*v_mean['vx']+v_mean['vy']*v_mean['vy']+v_mean['vz']*v_mean['vz']
    v_mean['drift'] = np.sqrt(v_mean['drift2'])

    # 3/2 k T = 1/2 m (<v²>-|<v>|²)
    temperature = mass/(3.*BOLTZMAN_CONSTANT)*(v_mean['v2']-v_mean['drift2'])
    # temperature = mass/(3.*BOLTZMAN_CONSTANT)*((v_mean['v']-v_mean['drift'])**2)

    return temperature

def variance_speed_evolution(data):
    data['v2'] = data['vx']*data['vx']+data['vy']*data['vy']+data['vz']*data['vz']
    data['v'] = np.sqrt(data['v2'])
    data_var = data.groupby(data.index).var()
    return data_var['v']


In [4]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
results_path = 'results/benchmarks/couette/monitoring.h5'
store = pd.HDFStore(results_path)
print(store.keys());

['/dsmc_collisions', '/dsmc_tracking', '/particles', '/wall_collisions']


In [15]:
dt = 1e-7
dy = 2.5e-5
dx = 2.5e-5
dz = 1e-3
particles_weight = 12500000000 # 6400000000
nb_cells = 40
# nb_particles_simu = 10000
mass = 2.16e-25 # kg


In [6]:
particles = store['particles']
indexes = np.unique(particles.index)

In [7]:
temp_evo = temperature_evolution(particles, mass)
speed_var = variance_speed_evolution(particles)

In [8]:
fig, ax = plt.subplots(1,2, constrained_layout = True)
ax[0].plot(indexes*dt, temp_evo)
ax[1].plot(indexes*dt, speed_var);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
frame = 3000
bins = 50
frame_df = particles.loc[particles.index == frame]
fig, ax = plt.subplots(2,2, constrained_layout = True)
ax[0,0].hist(frame_df['vx'],bins = bins,  color = 'r')
ax[0,1].hist(frame_df['vy'],bins = bins,  color = 'g')
ax[1,0].hist(frame_df['vz'],bins = bins,  color = 'b')
ax[1,1].hist(frame_df['v'], bins = bins, color = 'k');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
particles['idx_cell'] = (particles['y']/dy).astype(int)

In [30]:
groups_count = particles.groupby('idx_cell').count()

fig, ax = plt.subplots()
ax.plot(groups_count['v']/(dx*dy*dz*indexes.shape[0])*particles_weight, groups.index*dy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
groups = particles.groupby('idx_cell').mean()

In [ ]:
groups

In [25]:
fig, ax = plt.subplots()
ax.plot(groups['v'], groups.index*dy)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Choosing your simulations values 

In [10]:
def minimum_iterations(time_step_factor, target_accuracy, temperature, mass, caracteristic_speed, discretization_factor, total_number_particles):
    kb = 1.38e-23 
    return (time_step_factor*kb*temperature*total_number_particles)/(target_accuracy**2*mass*caracteristic_speed**2*total_number_particles)

In [11]:
time_step_factor = 4
target_accuracy = 5e-2
temperature = 300 # K
mass = 2.16e-25 # kg 
caracteristic_speed = np.linspace(1, 200, 200) 
mean_particle_per_cell = 100
nb_cells_x = 1
nb_cells_y = 40
discretization_factor = 40 # at leqst the number of cells
total_number_particles = nb_cells_x*nb_cells_y*mean_particle_per_cell # at least nb_cells_y

In [89]:
iterations = minimum_iterations(time_step_factor, target_accuracy, temperature, mass, caracteristic_speed, discretization_factor, total_number_particles)

In [90]:
fig, ax = plt.subplots()
ax.set_yscale('log')
ax.plot(caracteristic_speed, iterations);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
iterations[99]

3066.6666666666665

# Energie d'une loi unforme

In [44]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
# maxwellienne 
temperature = 300 # K
kb = 1.38e-23
nrj_maxwellian = 3/2*kb*temperature # J 

In [62]:
# for a uniform law
u_arr = np.linspace(1, 1000, 100)
mass = 2.16e-25 # kg 

def get_nrj_uniform_distrib(mass, u, dimensions = 1, nb_samples = int(1e5)):
    list_U = np.zeros((nb_samples, dimensions))
    for k in range(dimensions):
        list_U[:,k] = np.random.uniform(low = -u, high = u, size = nb_samples)
    
    U2 = np.linalg.norm(list_U, axis = 1)**2
    return 0.5*mass*np.mean(U2)

nrjs_uniform_1D = np.array([get_nrj_uniform_distrib(mass, u) for u in u_arr]) # 414 in 1D
nrjs_uniform_3D = np.array([get_nrj_uniform_distrib(mass, u, dimensions = 3) for u in u_arr]) # 239,7 in 3D


In [63]:
fig, ax = plt.subplots()
ax.axhline(y=nrj_maxwellian)
ax.plot(u_arr, nrjs_uniform_1D,'r');
ax.plot(u_arr, nrjs_uniform_3D,'g');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …